In [16]:
import h5py
import matplotlib
import matplotlib.pyplot as plt
# from habitat.utils.visualizations import maps
import matplotlib.colors as mcolors
from matplotlib.colors import LinearSegmentedColormap
import numpy as np
import os
import cv2
import ast
from scipy.spatial.transform import Rotation as R

In [92]:
def get_true_depth(depth):
    return 0.5 + 4.5*depth

In [91]:
dir_with_hdf5_datasets = '/media/cds-s/data/Datasets/Habitat/datasets_hdf5'

In [97]:
for filename_hdf5_dataset in os.listdir(dir_with_hdf5_datasets):
    file = h5py.File(os.path.join(dir_with_hdf5_datasets, filename_hdf5_dataset), 'r')
    output_path = dir_with_hdf5_datasets[:dir_with_hdf5_datasets.rfind('/')]+'/datasets/'+ \
                  filename_hdf5_dataset[:filename_hdf5_dataset.rfind('.')]
    
    rgb = file['rgb']
    depth = file['depth']
    semantic = file['semantic']
    objectgoal = file['objectgoal']
    compass = file['compass']
    gps = file['gps']
    heading = file['heading']
    top_down_map = file['top_down_map']
    fog_of_war_mask = file['fog_of_war_mask']
    agent_map_coord = file['agent_map_coord']
    agent_angle = file['agent_angle']
    mapping = np.array(file['mapping'])
    index_to_title_map = ast.literal_eval(str(np.array(file['index_to_title_map']))[2:-1])
    positions = file['position']
    rotations = file['rotation']
    depth = np.array(list(map(get_true_depth, depth)))
    
    os.makedirs(os.path.join(output_path,'rgb'), exist_ok=True)
    os.makedirs(os.path.join(output_path,'depth'), exist_ok=True)
    depth_file = open(os.path.join(output_path, 'depth.txt'), 'w')
    rgb_file = open(os.path.join(output_path, 'rgb.txt'), 'w')
    gt_filename = os.path.join('/media/cds-s/data/Datasets/Habitat/gt/',filename.split('/')[-1])
    gt_filename = gt_filename[:gt_filename.rfind('.')] + '.txt'
    gt_file = open(gt_filename, 'w')
    rgb_file.close()
    depth_file.close()
    gt_file.close()
    depth_file = open(os.path.join(output_path, 'depth.txt'), 'a')
    rgb_file = open(os.path.join(output_path, 'rgb.txt'), 'a')
    gt_file = open(gt_filename, 'a')
    for i in range(3):
        depth_file.write('ignore\n')
        rgb_file.write('ignore\n')
    for i in range(len(rgb)):
        cv2.imwrite(os.path.join(output_path, 'rgb', str(i).zfill(6)+'.png'), rgb[i])
        cv2.imwrite(os.path.join(output_path, 'depth', str(i).zfill(6)+'.exr'), depth[i])
        rgb_file.write('{:.3f} '.format(i/10))
        rgb_file.write('rgb/'+str(i).zfill(6)+'.png\n')
        depth_file.write('{:.3f} '.format(i/10))
        depth_file.write('depth/'+str(i).zfill(6)+'.exr\n')
        gt_file.write('{:.3f} {:.8e} {:.8e} {:.8e} {:.8e} {:.8e} {:.8e} {:.8e}\n'\
                      .format(i/10, -positions[i, 0], positions[i, 1], -positions[i, 2],
                        rotations[i, 0], rotations[i, 1], rotations[i, 2], rotations[i, 3]))
    rgb_file.close()
    depth_file.close()
    gt_file.close()